In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=73398b0ea38cb9640a364bf8ff05ece3f7f07ed83c4ab023e73f068070865732
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [3]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

In [4]:
os.chdir("/content/drive/My Drive/IRProject")

In [5]:
# Read the csv file
df = pd.read_csv('covid_df_10000.csv')
df

,Unnamed: 0,doc_id,title,abstract,text_body
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...
...,...,...,...,...,...
9995,9996,e6fe93e18fe7e74cc6d90086205e877c4245df35,Journal Pre-proof Targeted delivery of inhalab...,The coronavirus disease 2019 pandemic has led ...,Coronavirus disease 2019 (COVID-19) is highly ...
9996,9997,a6f37939b6a069def17ff724d2d49bbb41ce8f82,NaN,Governments around the globe have started to d...,"The novel coronavirus , first detected late 20..."
9997,9998,8126a598911ae325a6ebf1e1ca2fb9ae08cc73a2,NaN,NaN,as a means of preventing transmission (Chu et ...
9998,9999,cc3f4dc92499c431d05f71a9a4d0e74704f0a30b,Surveillance of acute SARS-CoV-2 infections in...,Background: Switzerland had one of the highest...,The role of schools in severe acute respirator...


In [6]:
# Concatenate the title and abstract columns to form a new column called 'full_text'
df['title_body'] = df['title'].fillna('') + ' ' + df['text_body'].fillna('')
df

,Unnamed: 0,doc_id,title,abstract,text_body,title_body
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...,"Long Term Predictors of Breathlessness, Exerci..."
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...,Emergence of universality in the transmission ...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...,Supporting families to protect child health: P...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...,Perspectives on the Early Quality of Evidence ...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...,Quantify the role of superspreaders -opinion l...
...,...,...,...,...,...,...
9995,9996,e6fe93e18fe7e74cc6d90086205e877c4245df35,Journal Pre-proof Targeted delivery of inhalab...,The coronavirus disease 2019 pandemic has led ...,Coronavirus disease 2019 (COVID-19) is highly ...,Journal Pre-proof Targeted delivery of inhalab...
9996,9997,a6f37939b6a069def17ff724d2d49bbb41ce8f82,NaN,Governments around the globe have started to d...,"The novel coronavirus , first detected late 20...","The novel coronavirus , first detected late 2..."
9997,9998,8126a598911ae325a6ebf1e1ca2fb9ae08cc73a2,NaN,NaN,as a means of preventing transmission (Chu et ...,as a means of preventing transmission (Chu et...
9998,9999,cc3f4dc92499c431d05f71a9a4d0e74704f0a30b,Surveillance of acute SARS-CoV-2 infections in...,Background: Switzerland had one of the highest...,The role of schools in severe acute respirator...,Surveillance of acute SARS-CoV-2 infections in...


In [7]:
# Load a pre-trained BERT model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Generate sentence embeddings for each full text in the DataFrame
embeddings = model.encode(df['title_body'].tolist(), convert_to_tensor=True)

In [8]:
# Define a function for searching the DataFrame using a query string
def search(query, embeddings, top_n=10):
    # Encode the query string to generate a query embedding
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the cosine similarity between the query embedding and the embeddings of each full text
    cos_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings).cpu().numpy()

    # Get the indices of the top n scores in descending order
    top_indices = cos_scores.argsort()[-top_n:][::-1]

    # Sort the cosine similarity scores array in a descending order
    cos_scores_sorted = np.sort(cos_scores)
    cos_scores_sorted_descending = cos_scores_sorted[::-1]

    # Return the DataFrame rows corresponding to the top n scores and the numpy array of the top n scores
    return df.iloc[top_indices], cos_scores_sorted_descending


In [9]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: vaccine effectiveness
Top 10 results for the query 'vaccine effectiveness':

Score: 0.6528316736221313
Title: pharmaceutics Nanoparticles to Improve the Efficacy of Vaccines
Body Text:
 A vaccine, providing active and protective immunity against a target disease, contains an agent that originated from and/or resembles a disease-causing microorganism. It is often made from a weakened or inactivated microbe, its toxins, or one of its nucleotides, peptides or proteins. Vaccines can be prophylactic to ameliorate or better prevent the effects of a wild-type pathogen, or therapeutic against likely cancers. To date, the World Health Organization lists twenty-seven preventable infections for which vaccines are available [1] -far less than what our society needs.
 Vaccination, a process of introducing foreign antigenic material(s) in order to activate a host immune system, has been a key strategy to control diseases and improve quality of life in humans and animals. Des

In [10]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: covid-19 transmission
Top 10 results for the query 'covid-19 transmission':

Score: 0.5683721899986267
Title: Targeting COVID-19 (SARS-CoV-2) main protease through active phytochemicals of ayurvedic medicinal plants -Withania somnifera (Ashwagandha), Tinospora cordifolia (Giloy) and Ocimum sanctum (Tulsi) -a molecular docking study
Body Text:
 As we pen down this sentence, cases of COVID-19 (Coronavirus disease 19) are continuously increasing across the globe. In accordance to 26 th July 2020 as per WHO (World Health Organization) situation report No. 187 and Centre for System Science and Engineering, Johns Hopkins University, globally greater than 15.9 million cases have been stated, resulting in 643,000 deaths, 9.22 million recovered cases with United States reporting the highest number of cases around 4.17 million (World Health Organization (WHO), 2020) (Johns Hopkins CSSE, 2020) . Speaking for India as per Ministry of Health and Family Welfare, till 26 th J

# Let us repeat the same steps for the 'abstract' column instead of the 'text_body' and see which method yields higher similarity scores.

In [11]:
# Concatenate the title and abstract columns to form a new column called 'full_text'
df['title_abstract'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')
df

,Unnamed: 0,doc_id,title,abstract,text_body,title_body,title_abstract
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...,"Long Term Predictors of Breathlessness, Exerci...","Long Term Predictors of Breathlessness, Exerci..."
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...,Emergence of universality in the transmission ...,Emergence of universality in the transmission ...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...,Supporting families to protect child health: P...,Supporting families to protect child health: P...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...,Perspectives on the Early Quality of Evidence ...,Perspectives on the Early Quality of Evidence ...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...,Quantify the role of superspreaders -opinion l...,Quantify the role of superspreaders -opinion l...
...,...,...,...,...,...,...,...
9995,9996,e6fe93e18fe7e74cc6d90086205e877c4245df35,Journal Pre-proof Targeted delivery of inhalab...,The coronavirus disease 2019 pandemic has led ...,Coronavirus disease 2019 (COVID-19) is highly ...,Journal Pre-proof Targeted delivery of inhalab...,Journal Pre-proof Targeted delivery of inhalab...
9996,9997,a6f37939b6a069def17ff724d2d49bbb41ce8f82,NaN,Governments around the globe have started to d...,"The novel coronavirus , first detected late 20...","The novel coronavirus , first detected late 2...",Governments around the globe have started to ...
9997,9998,8126a598911ae325a6ebf1e1ca2fb9ae08cc73a2,NaN,NaN,as a means of preventing transmission (Chu et ...,as a means of preventing transmission (Chu et...,
9998,9999,cc3f4dc92499c431d05f71a9a4d0e74704f0a30b,Surveillance of acute SARS-CoV-2 infections in...,Background: Switzerland had one of the highest...,The role of schools in severe acute respirator...,Surveillance of acute SARS-CoV-2 infections in...,Surveillance of acute SARS-CoV-2 infections in...


In [12]:
# Generate sentence embeddings for each full text in the DataFrame
embeddings = model.encode(df['title_abstract'].tolist(), convert_to_tensor=True)

In [13]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: vaccine effectiveness
Top 10 results for the query 'vaccine effectiveness':

Score: 0.7922271490097046
Title: To appear in: Vaccine
Body Text:
 The coronavirus disease 2019 (COVID- 19) pandemic and associated public health measures aiming to limit its transmission have had an unprecedented impact on healthcare services at all levels, including on routine childhood immunizations [1] [2] [3] . Multiple jurisdictions worldwide have reported significant declines in childhood vaccines administered and immunization coverage in the months following the pandemic declaration [4] [5] [6] [7] [8] [9] [10] [11] [12] [13] [14] . Disruptions to routine childhood immunization services leading to decreased coverage are concerning for possible resurgence of vaccine-preventable diseases (VPDs), including measles [15] [16] [17] .
 In response to these concerns, international and Canadian public health and governmental institutions have recommended that routine immunizations are e

In [14]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: covid-19 transmission
Top 10 results for the query 'covid-19 transmission':

Score: 0.8363113403320312
Title: Aspirin for COVID-19
Body Text:
 Thrombosis is a key feature of severe COVID-19, with 5-30% of hospitalised patients 61 (depending on illness severity) experiencing a major venous thromboembolic event 62 individuals involved in the trial were masked to aggregated outcome data during the 128 trial. 129
 A single online follow-up form was completed when participants were discharged, had 131 died or at 28 days after randomisation, whichever occurred earliest (appendix p 35-132 41). Information was recorded on adherence to allocated study treatment, receipt of 133 other COVID-19 treatments, duration of admission, receipt of respiratory or renal 134 support, and vital status (including cause of death). In addition, in the UK, routine 135 healthcare and registry data were obtained including information on vital status (with 136 date and cause of death), disch